# OTP분실안내 AI 고객센터 챗봇 만들기
## 작성자 : AISchool ( http://aischool.ai/%ec%98%a8%eb%9d%bc%ec%9d%b8-%ea%b0%95%ec%9d%98-%ec%b9%b4%ed%85%8c%ea%b3%a0%eb%a6%ac/ )
## 민원(콜센터) 질의-응답 데이터 다운받기 : https://www.aihub.or.kr/aihubdata/data/view.do?currMenu=115&topMenu=100&aihubDataSe=data&dataSetSn=98

# LangChain 라이브러리 설치

In [ ]:
!pip install langchain openai chromadb tiktoken pypdf unstructured sentence-transformers langchain-community

In [1]:
import pandas as pd

In [2]:
import os

print(os.getcwd())

/Users/hajuheon/PycharmProjects/aice/AI 쳇봇


# 데이터 읽어오기

In [5]:
json_file_path = './data/민원(콜센터) 질의응답_금융보험_사고 및 보상 문의_Training.json'
df = pd.read_json(json_file_path)
df

,도메인,카테고리,대화셋일련번호,화자,문장번호,고객의도,상담사의도,QA,고객질문(요청),상담사질문(요청),고객답변,상담사답변,개체명,용어사전,지식베이스
0,금융/보험,사고 및 보상 문의,A4075,고객,1,비밀번호 오류,,Q,인터넷뱅킹 로그인이 안돼요?,,,,"인터넷뱅킹, 로그인",인터넷뱅킹/ 금융서비스,"로그인,금융서비스"
1,금융/보험,사고 및 보상 문의,A4075,상담사,2,,비밀번호 오류,Q,,혹시 비밀번호를 잊어버리셨나요?,,,비밀번호,비밀번호/ 보안,"비밀번호,보안"
2,금융/보험,사고 및 보상 문의,A4075,고객,3,비밀번호 오류,,A,,,알고 있는 번호를 했는데 다 안된다고 나와요,,비밀번호,비밀번호/ 보안,"비밀번호,보안"
3,금융/보험,사고 및 보상 문의,A4075,상담사,4,,비밀번호 오류,Q,,비밀번호 5회이상 잘못 입력하셨나요?,,,"비밀번호, 5회",비밀번호/ 보안/ 5회/ 횟수,"5회,횟수"
4,금융/보험,사고 및 보상 문의,A4075,고객,5,비밀번호 오류,,A,,,어제 2번해서 안되서 오늘 다시 들어갔는데 안되요,,,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
84027,금융/보험,사고 및 보상 문의,A32567,상담사,16,,분실 등록 사고,Q,,혹시 언제 카드를 잃어버린건가요?,,,카드,,카드
84028,금융/보험,사고 및 보상 문의,A32567,고객,17,분실 등록 사고,,A,,,그걸 모르겠어요,,,,
84029,금융/보험,사고 및 보상 문의,A32567,상담사,18,,분실 등록 사고,A,,,,어머니께 바로 분실신고 해달라고 하세요,분실신고,분실신고/유실신고,"분실신고,유실신고"
84030,금융/보험,사고 및 보상 문의,A32567,고객,19,분실 등록 사고,,Q,분실신고는 전화로 되는거죠?,,,,"분실신고, 전화",전화/연락,"전화,연락"


In [6]:
len(df)

84032

In [16]:
df.columns

Index(['도메인', '카테고리', '대화셋일련번호', '화자', '문장번호', '고객의도', '상담사의도', 'QA',
       '고객질문(요청)', '상담사질문(요청)', '고객답변', '상담사답변', '개체명 ', '용어사전', '지식베이스'],
      dtype='object')

# 각 column별 unique한 값 확인하기

In [8]:
도메인_리스트 = df['도메인'].unique()
for idx, 도메인 in enumerate(도메인_리스트):
  print(idx, 도메인)

0 금융/보험


In [17]:
카테고리_리스트 = df['카테고리'].unique()
for idx, 카테고리 in enumerate(카테고리_리스트):
  print(idx, 카테고리)

0 사고 및 보상 문의


In [18]:
대화셋일련번호_리스트 = df['대화셋일련번호'].unique()
for idx, 대화셋일련번호 in enumerate(대화셋일련번호_리스트):
  print(idx, 대화셋일련번호)

0 A4075
1 A4076
2 A4079
3 A4080
4 A4081
5 A4082
6 A4083
7 A4086
8 A4088
9 A4089
10 A4104
11 A4129
12 A4132
13 A4153
14 A4156
15 A4252
16 A4253
17 A4254
18 A4255
19 A4256
20 A4257
21 A4258
22 A4259
23 A4260
24 A4261
25 A4262
26 A4263
27 A4264
28 A4265
29 A4266
30 A4267
31 A4268
32 A4269
33 A4270
34 A4271
35 A4272
36 A4273
37 A4274
38 A4275
39 A4276
40 A4277
41 A4278
42 A4279
43 A4280
44 A4281
45 A4282
46 A4283
47 A4284
48 A4285
49 A4286
50 A4287
51 A4288
52 A4289
53 A4290
54 A4291
55 A4292
56 A4293
57 A4294
58 A4295
59 A4296
60 A4297
61 A4298
62 A4299
63 A4300
64 A4301
65 A4302
66 A4303
67 A4304
68 A4305
69 A4306
70 A4307
71 A4308
72 A4309
73 A4310
74 A4311
75 A4312
76 A4313
77 A4314
78 A4315
79 A4316
80 A4317
81 A4318
82 A4319
83 A4320
84 A4321
85 A4322
86 A4323
87 A4324
88 A4325
89 A4326
90 A5028
91 A5051
92 A5054
93 A5141
94 A5146
95 A5147
96 A5148
97 A5154
98 A5173
99 A5174
100 A5175
101 A5190
102 A5192
103 A5193
104 A5194
105 A5195
106 A5199
107 A5203
108 A5204
109 A5205
110 A5206


In [ ]:
화자_리스트 = df['화자'].unique()
for idx, 화자 in enumerate(화자_리스트):
  print(idx, 화자)

In [ ]:
문장번호_리스트 = df['문장번호'].unique()
for idx, 문장번호 in enumerate(문장번호_리스트):
  print(idx, 문장번호)

In [ ]:
고객의도_리스트 = df['고객의도'].unique()
for idx, 고객의도 in enumerate(고객의도_리스트):
  print(idx, 고객의도)

In [ ]:
상담사의도_리스트 = df['상담사의도'].unique()
for idx, 상담사의도 in enumerate(상담사의도_리스트):
  print(idx, 상담사의도)

In [ ]:
QA_리스트 = df['QA'].unique()
for idx, QA in enumerate(QA_리스트):
  print(idx, QA)

In [ ]:
고객질문_요청_리스트 = df['고객질문(요청)'].unique()
for idx, 고객질문_요청 in enumerate(고객질문_요청_리스트):
  print(idx, 고객질문_요청)

In [ ]:
상담사질문_요청_리스트 = df['상담사질문(요청)'].unique()
for idx, 상담사질문_요청 in enumerate(상담사질문_요청_리스트):
  print(idx, 상담사질문_요청)

In [ ]:
고객답변_리스트 = df['고객답변'].unique()
for idx, 고객답변 in enumerate(고객답변_리스트):
  print(idx, 고객답변)

In [ ]:
상담사답변_리스트 = df['상담사답변'].unique()
for idx, 상담사답변 in enumerate(상담사답변_리스트):
  print(idx, 상담사답변)

In [ ]:
개체명_리스트 = df['개체명 '].unique()
for idx, 개체명 in enumerate(개체명_리스트):
  print(idx, 개체명)

In [ ]:
용어사전_리스트 = df['용어사전'].unique()
for idx, 용어사전 in enumerate(용어사전_리스트):
  print(idx, 용어사전)

In [ ]:
지식베이스_리스트 = df['지식베이스'].unique()
for idx, 지식베이스 in enumerate(지식베이스_리스트):
  print(idx, 지식베이스)

# 고객의도가 'OTP분실'인 데이터만 필터링하기

In [ ]:
filtered_df = df[(df['고객의도'] == 'OTP분실')]
filtered_df

In [ ]:
len(filtered_df)

# Vectorstore에 저장하기 위한 데이터 정제하기

In [ ]:
대화셋_set = set()
for count, (index, row) in enumerate(filtered_df.iterrows()):
    print(count, row['대화셋일련번호'], row['고객질문(요청)'])
    대화셋_set.add(row['대화셋일련번호'])

In [ ]:
대화셋_set

In [ ]:
filtered_df = df[df['대화셋일련번호'].isin(대화셋_set)]
filtered_df

In [ ]:
# '대화셋일련번호'로 그룹화하고 각 그룹을 인덱스 순서대로 정렬하여 원하는 컬럼들을 하나의 문자열로 합치기
def combine_columns(group):
    sorted_group = group.sort_index()  # 인덱스 순서대로 정렬
    combined_string = " ".join(
        sorted_group[['고객질문(요청)', '상담사질문(요청)', '고객답변', '상담사답변']].astype(str).values.flatten()
    )
    return combined_string

In [ ]:
combined_result = filtered_df.groupby('대화셋일련번호').apply(combine_columns).reset_index(name='combined_string')
combined_result

In [ ]:
combined_result[combined_result['대화셋일련번호']=='A28443']['combined_string'].values[0]

In [ ]:
temp_df = df[df['대화셋일련번호'] == 'A28443']
temp_df

In [ ]:
# 각 row의 값들을 순회하면서 출력
for index, row in combined_result.iterrows():
    print(f"대화셋일련번호: {row['대화셋일련번호']}, Combined String: {row['combined_string']}")

# 중복데이터 제거

*   'A28679' == 'A29168' == 'A6079' ('A29168', 'A6079' 제거)

In [ ]:
temp_df = df[df['대화셋일련번호'] == 'A28679']
temp_df

In [ ]:
temp_df = df[df['대화셋일련번호'] == 'A29168']
temp_df

In [ ]:
temp_df = df[df['대화셋일련번호'] == 'A6079']
temp_df

In [ ]:
combined_result

In [ ]:
# '대화셋일련번호'가 'A29168, 'A6079'인 행 삭제
combined_result = combined_result[combined_result['대화셋일련번호'] != 'A29168']
combined_result = combined_result[combined_result['대화셋일련번호'] != 'A6079']
combined_result

In [ ]:
# 각 row의 값들을 순회하면서 출력
for index, row in combined_result.iterrows():
    print(f"대화셋일련번호: {row['대화셋일련번호']}, Combined String: {row['combined_string']}")

# Document 생성 및 Vectorstore에 Embedding해서 저장

In [ ]:
from langchain.schema import Document

docs = []

# 각 row의 값들을 순회하면서 출력
for index, row in combined_result.iterrows():

    doc = Document(page_content=row['combined_string'], metadata={'대화셋일련번호': row['대화셋일련번호']})
    docs.append(doc)

In [ ]:
docs

In [ ]:
from langchain.embeddings import HuggingFaceEmbeddings

model_name = "jhgan/ko-sroberta-multitask" # (KorNLU 데이터셋에 학습시킨 한국어 임베딩 모델)
model_kwargs = {'device': 'cpu'}
encode_kwargs = {'normalize_embeddings': False}
embedding_model = HuggingFaceEmbeddings(
    model_name=model_name,
    model_kwargs=model_kwargs,
    encode_kwargs=encode_kwargs
)

In [ ]:
from langchain.vectorstores import Chroma

vectorstore = Chroma.from_documents(documents=docs, embedding=embedding_model)

# Sanity Check

In [ ]:
retriever = vectorstore.as_retriever()

In [ ]:
retrieved_docs = retriever.invoke(
    "OTP를 분실했는데 어떻게 하나요?"
)
print(retrieved_docs[0].page_content)

In [ ]:
retrieved_docs = retriever.invoke(
    "OTP를 재발급 받으려면 어떻게해야 하나요?"
)
print(retrieved_docs[0].page_content)

# LLM과 연동

##  OpenAI API Key 설정

In [ ]:
OPENAI_KEY = "여러분의_OPENAI_API_KEY"

In [ ]:
from langchain.chat_models import ChatOpenAI

llm = ChatOpenAI(model_name="gpt-4o", temperature=0, openai_api_key=OPENAI_KEY)

# 대화 history 저장을 위한 ConversationBufferMemory 설정

**ConversationBufferMemory** : 모든 대화내역을 저장

In [ ]:
from langchain.memory import ConversationBufferMemory

memory = ConversationBufferMemory()

In [ ]:
memory.load_memory_variables({})

In [ ]:
def load_memory(input):
    return memory.load_memory_variables({})["history"]

# 최종 Prompt 및 Chain 생성

In [ ]:
from langchain.prompts import PromptTemplate

template = """다음과 같은 맥락과 채팅히스토리를 사용하여 마지막 질문에 대답하십시오.
맥락: {context}
채팅히스토리: {history}
질문: {question}
도움이 되는 답변:"""
rag_prompt_custom = PromptTemplate.from_template(template)

In [ ]:
# RAG chain 설정
from langchain.schema.runnable import RunnablePassthrough
from langchain_core.prompts import MessagesPlaceholder

rag_chain = {"context": retriever, "question": RunnablePassthrough(), "history": load_memory} | rag_prompt_custom | llm

# Tracking을 위한 LangSmith 설정(Optional)

In [ ]:
import os
from uuid import uuid4

unique_id = uuid4().hex[0:8]
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_PROJECT"] = f"Tracing Walkthrough - {unique_id}"
os.environ["LANGCHAIN_ENDPOINT"] = "https://api.smith.langchain.com"
os.environ["LANGCHAIN_API_KEY"] = "여러분의_LangSmith_API_key"

In [ ]:
memory.load_memory_variables({})["history"]

In [ ]:
def get_result_and_save_memory(user_question):
  chat_result = rag_chain.invoke(user_question)

  # memory에 저장
  memory.chat_memory.add_user_message(user_question)
  memory.chat_memory.add_ai_message(chat_result.content)

  return chat_result.content

# OTP분실안내 AI 고객센터 챗봇 성능 evaluation (정성적 평가)

In [ ]:
filtered_df = df[(df['고객의도'] == 'OTP 분실')]
filtered_df

In [ ]:
temp_df = df[df['대화셋일련번호'] == 'A18526']
temp_df

In [ ]:
llm_response = get_result_and_save_memory("OTP를 잃어버렸어요.")
llm_response

In [ ]:
memory.load_memory_variables({})["history"]

In [ ]:
llm_response = get_result_and_save_memory("제가 여러 은행을 이 한 OTP로 이용중이었는데 분실신고하면 이제 모든 은행에서 다 이용이 안되는거죠?")
llm_response

In [ ]:
memory.load_memory_variables({})["history"]

In [ ]:
llm_response = get_result_and_save_memory("그렇군요. 일때문에 바빠서 방문이 어려운데 혹시 모바일로 이용하는 OTP같은 건 없나요?")
llm_response

In [ ]:
memory.load_memory_variables({})["history"]

In [ ]:
# 대화내역을 저장한 Memory 초기화하기
# memory.clear()